In [81]:
# we load the things!

from ngsolve import *
#from netgen.csg import * 
from netgen.geom2d import SplineGeometry
from ngsolve.meshes import MakeStructured3DMesh

from netgen.occ import *
#from netgen.meshing import *

import scipy.sparse as sp
from scipy.optimize import curve_fit
import numpy as np

import pandas as pd

In [82]:
# some helper functions
def logspace_custom_decades(start, stop, points_per_decade):
    
    result = []
    current_decade = start
    while current_decade < stop:
        decade_points = np.logspace(np.log10(current_decade), np.log10(current_decade * 10), points_per_decade, endpoint=False)
        result.extend(decade_points)
        current_decade *= 10
    return np.array(result)

In [83]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

def curl_a_times_b(a,b):
    """ Returns curl(a) x b """
    a_Jac = Grad(a)
    
    c0 = a_Jac[2, 1] - a_Jac[1, 2]  # (curl(a))[0]
    c1 = a_Jac[0, 2] - a_Jac[2, 0]  # (curl(a))[1]
    c2 = a_Jac[1, 0] - a_Jac[0, 1]  # (curl(a))[2]      

    cxb0 = c1 * b[2] - c2 * b[1]
    cxb1 = c2 * b[0] - c0 * b[2]
    cxb2 = c0 * b[1] - c1 * b[0]

    curlAtimesB = CF((cxb0, cxb1, cxb2))

    return curlAtimesB

In [84]:
# Functions for plotting, linear regression fit line for convergence

def reference_line_func(h_values, scaling_factor, slope):

    return scaling_factor * h_values ** slope

def fit_reference_line(h_values, error_values):

    popt, _ = curve_fit(reference_line_func, h_values, error_values, p0=[1, 1])

    scaling_factor, slope = popt
    return scaling_factor, slope

In [85]:
# Functions to calculate h_max

def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    # Create the Cayley-Menger matrix (5x5)
    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2  
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)

    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_hmax(mesh):
    max_h = 0 
    if mesh.dim == 2:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

In [86]:
def createGeometry(hmax):

    largeBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0.5, 0.5, 0.5))
    smallBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0, 0, 0))

    reentrantCornerGeo3D = largeBrick - smallBrick
    # Here instead of the following mesh command, I think I need to implement a structured mesh with the manual mesh generation approach.
    mesh = Mesh(OCCGeometry(reentrantCornerGeo3D).GenerateMesh(maxh=hmax))

    return mesh

def createStructuredGeometry(n):
    mesh =  MakeStructured3DMesh(hexes=False, nx=n, ny=n, nz=n)
    return mesh



In [87]:
# def hodgeLaplace1Forms(mesh,
#                        g = None, # this is the manufactured solution, when none is given we set it to the zero solution
#                        order = 1,
#                        C_w = 1):
    
#     if g is None:
#         g = CF((0,0,0))

#     h_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
#     h_1 = H1(mesh, order=order)     # For 0-forms, H1 space
#     fes = h_curl * h_1
#     (u, p), (v, q) = fes.TnT()

#     B = BilinearForm(fes)

#     B += u * grad(q) * dx
#     B += - p * q * dx
    
#     B +=  grad(p) * v * dx
#     B +=  curl(u) * curl(v) * dx

#     n = specialcf.normal(mesh.dim)
#     t = specialcf.tangential(mesh.dim)
#     h = specialcf.mesh_size

#     ### TILL HERE ALL IS GOOD ###

#     gamma_p_v = v - n*(v*n)
#     gamma_p_u = u - n*(u*n)
#     gamma_n_u = Cross(n, curl(u))
#     gamma_n_v = Cross(n, curl(v))

#     curlu = curl(u)
#     curlv = curl(v)
#     curl_u_times_v = Cross(curlu, v)
#     curl_v_times_u = Cross(curlv, u)

#     B += curl_u_times_v * n * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
#     B += curl_v_times_u * n * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
#     B += C_w/h * u.Trace() * v.Trace() * ds
    
#     f = CF(GCurl(GCurl(g)) - GGrad(GDiv(g))) 

#     rhs = LinearForm(fes)
#     rhs += f * v * dx

#     gamma_p_u_m = g - n*(g*n)
#     #curl_v_times_u_m = CF((curlv[1]*g[2] - curlv[2]*g[1], curlv[2]*g[0] - curlv[0]*g[2], curlv[0]*g[1] - curlv[1]*g[0]))
#     curl_v_times_u_m = Cross(curlv, g)
#     rhs += (C_w / h) * gamma_p_u_m * gamma_p_v * ds(skeleton=True)
#     #rhs +=  gamma_p_u_m  * gamma_n_v * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
#     #rhs += (C_w / h) * gamma_p_u_m * v.Trace() * ds
#     rhs += curl_v_times_u_m * n * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
#     rhs +=  g * n * q * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

#     with TaskManager():
#         B.Assemble()
#         rhs.Assemble()
#         sol = GridFunction(fes)
#         res = rhs.vec-B.mat * sol.vec
#         inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
#         sol.vec.data += inv * res

#     gf_u , gf_p = sol.components

#     #curl_u = CF(Grad(gf_u)[1,0] - Grad(gf_u)[0,1])
#     curl_u = curl(gf_u)
#     grad_p = grad(gf_p)

#     curl_u_m = CF(GCurl(g))
#     p_m = - CF(GDiv(g))
#     grad_p_m = CF(GGrad(p_m))

#     gf_gamma_p_u = CF(gf_u - n*(gf_u*n))
#     gf_gamma_p_u_m = CF(g - n*(g*n))

#     gf_gamma_n_u = BoundaryFromVolumeCF(Cross(n, curl_u))
#     gf_gamma_n_u_m = BoundaryFromVolumeCF(Cross(n, curl_u_m))

#     gf_u_n = CF(gf_u * n)
#     gf_u_n_m = CF(g * n)

#     L2_error_u = sqrt(Integrate((gf_u - g)**2, mesh))
#     L2_error_curl_u = sqrt(Integrate((curl_u - curl_u_m)**2, mesh))
#     L2_error_p = sqrt(Integrate((gf_p - p_m)**2, mesh))
#     L2_error_grad_p = sqrt(Integrate((grad_p - grad_p_m)**2, mesh))

#     L2_error_gamma_p_u = sqrt(Integrate((gf_gamma_p_u - gf_gamma_p_u_m)**2 * ds(skeleton=True, definedon=mesh.Boundaries(".*")), mesh))
#     L2_error_u_n = sqrt(Integrate((gf_u_n - gf_u_n_m)**2 * ds(skeleton=True, definedon=mesh.Boundaries(".*")), mesh))
#     L2_error_p_gamma = sqrt(Integrate((gf_p - p_m)**2 * ds(skeleton=True, definedon=mesh.Boundaries(".*")), mesh))

#     # terms for the hashtag error!
#     h_avg = 1 / Integrate(1, mesh, VOL) * Integrate(h, mesh, VOL)

#     HT_error_gamma_p = Integrate(h_avg**(-1)*(gf_gamma_p_u - gf_gamma_p_u_m)**2 * ds(skeleton=True, definedon=mesh.Boundaries(".*")), mesh)
#     HT_error_gamma_n = Integrate(h_avg*(gf_gamma_n_u - gf_gamma_n_u_m)**2 * ds(skeleton=True, definedon=mesh.Boundaries(".*")), mesh)
#     HT_error_u = sqrt(L2_error_u**2 + L2_error_curl_u**2 + HT_error_gamma_p + HT_error_gamma_n) 
#     #HT_error_u = HT_error_gamma_n

#     return fes.ndof, Norm(res), L2_error_u, L2_error_curl_u, L2_error_p, L2_error_grad_p, L2_error_gamma_p_u, L2_error_u_n, L2_error_p_gamma, HT_error_u, sqrt(HT_error_gamma_n), sqrt(HT_error_gamma_p)

In [ ]:
def hodgeLaplace1Forms(mesh,
                       g = None, # this is the manufactured solution, when none is given we set it to the zero solution
                       order = 1,
                       C_w = 1):
    
    if g is None:
        g = CF((0,0,0))

    h_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
    h_1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = h_curl * h_1
    (u, p), (v, q) = fes.TnT()

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size

    curl_u_x_v = curl_a_times_b(u, v)
    curl_v_x_u = curl_a_times_b(v, u)
    curl_v_x_g = curl_a_times_b(v, g)

    n_x_curl_u = Cross(n, curl(u)) 
    n_x_curl_v = Cross(n, curl(v))
    
    f = CF(GCurl(GCurl(g)) - GGrad(GDiv(g)))

    B = BilinearForm(fes)
    F = LinearForm(fes)

    B += curl(u) * curl(v) * dx
    B += v * grad(p) * dx

    B += u * grad(q) * dx
    B += - p * q * dx

    B += C_w / h * u * v * ds(skeleton=True)
    B += n_x_curl_u * v * ds(skeleton=True)
    B += n_x_curl_v * u * ds(skeleton=True)
   
    F += f * v * dx
    F += C_w / h * g * v * ds(skeleton=True)
    F += n_x_curl_v * g * ds(skeleton=True)
    F += g * n * q * ds(skeleton=True)

    with TaskManager():
        B.Assemble()
        F.Assemble()
        sol = GridFunction(fes)
        res = F.vec-B.mat * sol.vec
        inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
        sol.vec.data += inv * res

    gf_u , gf_p = sol.components

    curl_u = curl(gf_u)
    grad_p = grad(gf_p)

    curl_g = CF(GCurl(g))
    p_m = - CF(GDiv(g))
    grad_p_m = CF(GGrad(p_m))

    # gf_gamma_p_u = CF(gf_u - n*(gf_u*n))
    # gf_gamma_p_g = CF(g - n*(g*n))
    gf_gamma_p_u = CF(Cross(n, Cross(gf_u, n)))
    gf_gamma_p_g = CF(Cross(n, Cross(g, n)))

    gf_gamma_n_u = BoundaryFromVolumeCF(Cross(n, curl_u))
    gf_gamma_n_u_m = BoundaryFromVolumeCF(Cross(n, curl_g))

    gf_u_n = CF(gf_u * n)
    gf_g_n = CF(g * n)

    L2_error_u = sqrt(Integrate((gf_u - g)**2, mesh))
    L2_error_curl_u = sqrt(Integrate((curl_u - curl_g)**2, mesh))
    L2_error_p = sqrt(Integrate((gf_p - p_m)**2, mesh))
    L2_error_grad_p = sqrt(Integrate((grad_p - grad_p_m)**2, mesh))

    L2_error_gamma_p_u = sqrt(Integrate((gf_gamma_p_u - gf_gamma_p_g)**2 * ds(skeleton=True), mesh))
    L2_error_u_n = sqrt(Integrate((gf_u_n - gf_g_n)**2 * ds(skeleton=True), mesh))
    L2_error_p_gamma = sqrt(Integrate((gf_p - p_m)**2 * ds(skeleton=True), mesh))

    # terms for the hashtag error
    h_avg = 1 / Integrate(1, mesh, VOL) * Integrate(h, mesh, VOL)

    HT_error_gamma_p = Integrate(h_avg**(-1)*(gf_gamma_p_u - gf_gamma_p_g)**2 * ds(skeleton=True), mesh)
    HT_error_gamma_n = Integrate(h_avg*(gf_gamma_n_u - gf_gamma_n_u_m)**2 * ds(skeleton=True), mesh)
    HT_error_u = sqrt(L2_error_u**2 + L2_error_curl_u**2 + HT_error_gamma_p + HT_error_gamma_n) 
    #HT_error_u = HT_error_gamma_n

    return fes.ndof, Norm(res), L2_error_u, L2_error_curl_u, L2_error_p, L2_error_grad_p, L2_error_gamma_p_u, L2_error_u_n, L2_error_p_gamma, HT_error_u, sqrt(HT_error_gamma_n), sqrt(HT_error_gamma_p)

In [89]:
# definition of manufactured solutions

#omega_m_sinusoids_2D = CF((-sin(pi*x)*cos(pi*y), cos(pi*x)*sin(pi*y))) # This one vanishes
#omega_m_sinusoids_2D = CF((sin(pi*x)*sin(pi*y), sin(pi*x)*sin(pi*y))) # This one vanishes!

omega_m_sinusoids_3D = CF(( -cos(x)*sin(y)*cos(z), 
                            sin(x)*cos(y)*sin(z),
                            -cos(x)*sin(y)*cos(z)))

#omega_m_sinusoids_3D = CF((z**2, 12*cos(1/3*x), sin(x)+2*y**4))


In [90]:
# Convergence study data generation for 2D

saveBigCSV = True

refinementVals = [6, 8, 10]
h_max_init = 0.5

maxh_values = [] 

Cw_vals = logspace_custom_decades(10, 100, 20)
#Cw_vals = np.linspace(2, 11, 10)
#Cw_vals = [10]

orders = [1, 2]

all_results = []

for refinementVal in refinementVals:
    mesh = createStructuredGeometry(refinementVal)
    h_max_eval = calc_hmax(mesh)
    maxh_values.append(h_max_eval)
    print("doing h_max: ", h_max_eval)

    for order_cw in orders:
        results_cw = []
        print("doing order: ", order_cw)

        for C_w in Cw_vals:
            g = omega_m_sinusoids_3D

            ndof, res, L2_error_u, L2_error_curl_u, L2_error_p, L2_error_grad_p, L2_error_gamma_p_u, L2_error_u_n, L2_error_p_Gamma, HT_error_u, HT_error_gamma_n, HT_error_gamma_p = hodgeLaplace1Forms(
                mesh, g=g, order=order_cw, C_w=C_w
            )

            row_dict = {
                'order': order_cw,
                'hmax': h_max_eval,
                'C_w': C_w,
                'ndof': ndof,
                'res': res,
                'L2_error_u': L2_error_u,
                'L2_error_curl_u': L2_error_curl_u,
                'L2_error_p': L2_error_p,
                'L2_error_grad_p': L2_error_grad_p,
                'L2_error_gamma_p_u': L2_error_gamma_p_u,
                'L2_error_u_n': L2_error_u_n,
                'L2_error_p_Gamma': L2_error_p_Gamma,
                'HT_error_u': HT_error_u,
                'HT_error_gamma_n': HT_error_gamma_n,
                'HT_error_gamma_p': HT_error_gamma_p
            }
            all_results.append(row_dict)

df_all_results = pd.DataFrame(all_results)

if (saveBigCSV == True):
    df_all_results.to_csv("all_3D_1forms_simulation_results.csv", index=False)

doing h_max:  0.14433756729740652
doing order:  1
doing order:  2
doing h_max:  0.10825317547305482
doing order:  1
doing order:  2
doing h_max:  0.08660254037844395
doing order:  1
doing order:  2
